In [1]:
import json
import sqlite3
from pprint import pprint

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        In your assignment, this file is supposed to be a .py file.
        It is a .ipynb file here so that I can include extended explanations.
    </p>
</div>

## Step 1.

Open the JSON file and dig around.
What we're looking for is the basic entities, how their structure might vary, the formatting of dates/timestamps/datetimes, and anything else that might cause problems for us.


In [2]:
with open("reids.json") as f:
    data = json.load(f)

When poking around (very) large JSON files, it's better to look at types and keys first:

In [3]:
type(data)

dict

In [4]:
data.keys()

dict_keys(['orders'])

This tells us that the "root" object we got back from the "API" is a JSON object.
This isn't uncommon.
We could have gotten a List back as well.
We can look at the type of "orders":

In [5]:
type(data["orders"])

list

This is where our data is...how much of it is there?

In [6]:
len(data["orders"])

2071

We can put it in a convenience variable and then look at a few.
`pprint` with compact=True will make the output look prettier...

In [7]:
orders = data["orders"]

In [8]:
pprint(orders[0:5], compact=True)

[{'charges': {'date': '04/01/21 11:16',
              'subtotal': 2.75,
              'taxes': 0.19,
              'total': 2.94},
  'items': [{'name': 'coffee', 'price': 2.75}],
  'payment': {'card_type': 'mastercard',
              'cardholder': 'Brent Morgan',
              'last_4_card_number': '5056',
              'method': 'credit_card',
              'zip': '21274'}},
 {'charges': {'date': '04/01/21 11:10',
              'subtotal': 4.0,
              'taxes': 0.28,
              'total': 4.28},
  'items': [{'name': 'coffee', 'price': 2.75},
            {'name': 'espresso', 'price': 1.25}],
  'payment': {'card_type': 'visa',
              'cardholder': 'Andrew Luna',
              'last_4_card_number': '6072',
              'method': 'credit_card',
              'zip': '21213'}},
 {'charges': {'date': '04/01/21 8:34',
              'subtotal': 8.5,
              'taxes': 0.6,
              'total': 9.1},
  'items': [{'name': 'coffee', 'price': 2.75},
            {'name': 'flavo

Here we can see a number of things:

1. Each order has a charge, items, and payment.
2. The date for the charge is in the format %m/%d/%y but the time is unusual. The hour is not zero padded. Because the meridian or time zone is not indicated, we can assume, for now, that it is a 24 hour clock and local time. If we were using a timezone sensitive database, we'd need to take that into account.
3. We can see at least two payment methods. Are there more? Since we lack documentation, we need to explore.

In [9]:
set([order["payment"]["method"] for order in orders])

{'cash', 'credit_card'}

We can't see any other payment methods but that doesn't mean there aren't any.
This is a common problem in datawarehousing when working with 3rd party vendors who either don't or won't document their data (sometimes because they don't want you going off their platform).
If we discover a new payment method later, we might need to redo this table.
Since we can see the format of `cash` and `credit_card` above, we can plan the table from there.
If we couldn't we'd need to find an example of each.

It's also worth noting that APIs differ in how they treat "NULL".
For example, we have:

```
{'card_type': 'visa',
 'cardholder': 'Tami Hunt',
 'last_4_card_number': '9791',
 'method': 'credit_card',
 'zip': '21270'}
```
but
```
{'method': 'cash'}
```
An alternative might have been:
```
{'card_type': None,
 'cardholder': None,
 'last_4_card_number': None,
 'method': 'cash',
 'zip': None}
```

The layout of each table seems straightforward:

* charges
    * id (Integer, Primary Key)
    * date (String in SQLite, Datetime in other DBs)
    * subtotal (Numeric)
    * taxes (Numeric)
    * total (Numeric)
    * payment_id (Integer, Foreign Key)
* items
    * id (Integer, Primary Key)
    * charge_id (Integer, Foreign Key)
    * name (String)
    * price (Numeric)
* payments
    * id (Integer, Primary Key)
    * method (String, one of "cash" or "credit_card")
    * card_type (String)
    * cardholder (String)
    * last_4_card_number (String)
    * zip (String)

It's tempting to think there should be an order table but it would just be ids (really, the charge table *is* the order table).
As mentioned previously, it's tempting to create a menu_items table but you have to be careful to preserve historicity (which is often the opposing goal of a production RDBMS...then we want the value to change everywhere when it's changed anywhere).
If one does add a menu_items table, it should only be the name of the item and perhaps it's type (drink, food, upcharge|addon).

<div style="background: peachpuff; margin:20px; padding: 20px;">
    <strong>Rubric</strong>
    <ul>
        <li>You don't need to have this exact schema but you should be close.</li>
        <li>This code must be in a .py file, as requested.</li>
    </ul>
</div>

As a result of your investigation, the `reids.sql` file should look, more or less, like this:

```
DROP TABLE IF EXISTS charges;
CREATE TABLE charges (
    id INTEGER PRIMARY KEY,
    date STRING,
    subtotal NUMERIC,
    taxes NUMERIC,
    total NUMERIC,
    payment_id INTEGER
);

DROP TABLE IF EXISTS items;
CREATE TABLE items (
    id INTEGER PRIMARY KEY,
    charge_id INTEGER,
    name STRING,
    price NUMERIC
);

DROP TABLE IF EXISTS payments;
CREATE TABLE payments (
    id INTEGER PRIMARY KEY,
    method STRING,
    card_type STRING,
    cardholder STRING,
    last_4_card_number STRING,
    zip STRING
);
```
If the placement of your foreign key fields is different, don't fret.
They are conventionally either at the start or the end of the table.

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Whether or not you actually declare fields to be <tt>PRIMARY KEY</tt> or <tt>FOREIGN KEY</tt> will depend on the use case.
        Most datawarehouse operations are large bulk operations.
        In many cases, you actually have to rebuild the entire datawarehouse each night because applications do not use updated_at fields.
        Key constraints, indexes, and the like can slow down build inserts and violations can cause multi-hour processes to fail.
        We assume that the application database has maintained all the relationships correctly or, at worst, that getting the data is more important than data integrity.
        The goals are different.
        The datawarehouse is not the System of Record.
    </p>
    <p>
        In this example, we only declare the <tt>PRIMARY KEY</tt>.
        In some cases, you may not even want to declare the PRIMARY KEY.
        For foreign keys, we use conventions for designating foreign keys ([tablename]_id).
    </p>
</div>

<div style="background: peachpuff; margin:20px; padding: 20px;">
    <strong>Rubric</strong>
    <ul>
        <li>Your datawarehouse DDL must be in a .sql file.</li>
        <li>If you included explicit Foreign Key declarations, that's fine. Just be aware that in datawarehouse <i>engineering</i>, you sometimes don't (as opposed to application engineering).</li>
    </ul>
</div>

Additionally, it might be worth splitting up date and time in some cases and with some databases.

## Step 2.

Put the data in the database.

We're hampered here by the lack of vendor provided primary and secondary keys.
This does sometimes happen in the real world, especially if you are pulling data directly from a report rather than a fully supported "Data" API.

The strategy would be something like this:

1. create and keep a counter for charges.
2. create and keep a counter for items. Add the charge_id to every item.
3. create and keep a counter for payment. Only increment the payment id if the payment hasn't been seen before.

This leads to an interesting question...is there only *one* cash payment or is every cash payment unique?

1. We could have a single cash payment. There's nothing that really distinguishes one from another. If we add a loyalty program in the future, the cash is still unique and we would add a loyalty number to the order/charge.
2. We could treat cash payments as unique. We might start asking for emails or other information, if people wanted receipts emailed instead of printed. In that case, there would be more information than just "cash".

Because this is datawarehouse, we'll go with a single cash payment for now and change things around if we need to do so.

How much "programming" v. "scripting" you do at this step is up to you.
It's often to maintain "programming".
We'll take the programming route here.
We need functions that takes in raw data, any foreign keys, and returns Tuples because that's what SQL uses for writing.

Let's write a generic function that takes a list of properties, raw data, and additional values and returns a Tuple of those property's values.

In [10]:
def extract_properties(properties, data, defaults=dict()):
    result = []
    for prop in properties:
        if prop in data:
            result.append(data[prop])
        else:
            result.append(defaults.get(prop, None)) # we do need things to be Nullable
    return tuple(result)

Now the fun begins...the main question is, what order should we do this in? And who should be responsible for checking and incrementing ids?

1. The only key we need to unique is payments. We need to keep a lookup of payment (cardholder, last_4_card_number) and ids, if that cardholder/card number has already been seen, get the id; otherwise generate a new one.
2. Go on to charges, using the payment_id, create a charge_id.
3. Go on to items, using the charge_id. Create item ids.

In [11]:
def process_payment(payment_data, payments, current_id):
    key = extract_properties(["cardholder", "last_4_card_number"], payment_data)
    if key in payments:
        return current_id, payments[key], None
    payments[key] = current_id
    payment_id = payments[key]
    current_id +=1
    return current_id, payment_id, extract_properties(["id", "method", "card_type", "cardholder", "last_4_card_number", "zip"], payment_data, {"id": payment_id})

This payment hasn't been seen before:

In [12]:
process_payment(orders[0]["payment"], {}, 1)

(2, 1, (1, 'credit_card', 'mastercard', 'Brent Morgan', '5056', '21274'))

This one has, so no need to write it to the database, but we do need to know its id:

In [13]:
process_payment(orders[0]["payment"], {("Cheryl Colon", '4166'): 1}, 2)

(3, 2, (2, 'credit_card', 'mastercard', 'Brent Morgan', '5056', '21274'))

In [14]:
def process_charge(charge, current_id, payment_id):
    result = extract_properties(["id", "date", "subtotal", "taxes", "total", "payment_id"], charge, {"id": current_id, "payment_id": payment_id})
    current_id += 1
    return current_id, result

In [15]:
process_charge(orders[0]["charges"], 1, 27)

(2, (1, '04/01/21 11:16', 2.75, 0.19, 2.94, 27))

items is going to be somewhat different because there are multiple items (and so we'll be returning a List of Tuples).

In [16]:
def process_items(items, current_id, charge_id):
    results = []
    for item in items:
        result = extract_properties(["id", "charge_id", "name", "price"], item, {"id": current_id, "charge_id": charge_id})
        results.append(result)
        current_id += 1
    return current_id, results

In [17]:
process_items(orders[0]["items"], 1, 7)

(2, [(1, 7, 'coffee', 2.75)])

Now we can piece these all together.
We generally like to make sure we properly parse all the data before we start writing it to the database but either way is fine.
In this example, we parse all the data first.

In [18]:
payment_id = 1
payment_lookup = {}
payments = []
charge_id = 1
charges = []
item_id = 1
items = []

In [19]:
for order in orders:
    payment_id, this_payment_id, payment = process_payment(order["payment"], payment_lookup, payment_id)
    if payment: # could be None
        payments.append(payment)
    charge_id, charge = process_charge(order["charges"], charge_id, this_payment_id)
    charges.append(charge)
    item_id, these_items = process_items(order["items"], item_id, charge_id)
    items += these_items # list concatenation

We can do some basic sanity checks:

In [20]:
len(payments)

757

In [21]:
len(set([extract_properties(["cardholder", "last_4_card_number"], order["payment"], {}) for order in orders]))

757

In [22]:
len(charges)

2071

In [23]:
len(orders)

2071

In [24]:
len(items)

4145

In [25]:
sum([len(order["items"]) for order in orders])

4145

(We could turn these into tests to make sure the script is working properly, if we automate it for some reason.)

Now we need to write everything to the DB.
First we make sure we're starting "fresh":

In [26]:
!rm reids.db

In [27]:
!sqlite3 reids.db < reids.sql

Next we get a connection and start inserting the data:

In [28]:
con = sqlite3.connect('reids.db')

In [29]:
con.executemany('INSERT INTO payments (id, method, card_type, cardholder, last_4_card_number, zip) VALUES (?, ?, ?, ?, ?, ?)', payments);
con.commit();

More sanity checks.
**Note**`execute` returns a Cursor to a result set. It's generator that needs to be realized somehow.
One way is to wrap it in list().

In [30]:
list(con.execute("SELECT COUNT(*) FROM payments"))

[(757,)]

In [31]:
con.executemany('INSERT INTO charges (id, date, subtotal, taxes, total, payment_id) VALUES (?, ?, ?, ?, ?, ?)', charges);
con.commit();

In [32]:
list(con.execute("SELECT COUNT(*) FROM charges"))

[(2071,)]

In [33]:
con.executemany('INSERT INTO items (id, charge_id, name, price) VALUES (?, ?, ?, ?)', items);
con.commit();

In [34]:
list(con.execute("SELECT COUNT(*) FROM items"))

[(4145,)]

And with that, you might switch to the command line to run some queries to poke at the data.

You might also come back here and decide that you want to add a `calendar` table which is a common thing to do.
This would have dates and indicate any closings, perhaps day of the week and holidays.

```
CREATE TABLE calendar (
    date STRING,
    day_of_week STRING
)
```

In [35]:
from datetime import datetime, timedelta

In [36]:
days_of_the_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

In [37]:
calendar = []

today = datetime(2021, 3, 31, 0, 0)
for _ in range(0, 61):
    today = today + timedelta(days=1)
    date = today.strftime("%m/%d/%y")
    day = days_of_the_week[today.weekday()]
    calendar.append((date, day))

In [38]:
con.executemany('INSERT INTO calendar (date, day_of_week) VALUES (?, ?)', calendar);
con.commit()

In [39]:
list(con.execute("SELECT * FROM calendar LIMIT 10"))

[('04/01/21', 'Thursday'),
 ('04/02/21', 'Friday'),
 ('04/03/21', 'Saturday'),
 ('04/04/21', 'Sunday'),
 ('04/05/21', 'Monday'),
 ('04/06/21', 'Tuesday'),
 ('04/07/21', 'Wednesday'),
 ('04/08/21', 'Thursday'),
 ('04/09/21', 'Friday'),
 ('04/10/21', 'Saturday')]

**Notes**

It's often tempting when developing scripts like this to comment all the tests and sanity checks out along all the "poking around".
You really should keep these tests and sanity checks in the script and fail when they fail.
Of course, this means you should also monitor your scripts.


<div style="background: peachpuff; margin:20px; padding: 20px;">
    <strong>Rubric</strong>
    <ul>
        <li>All of your ETL code should be in this .py file.</li>
        <li>You don't need to have made the calendar table. It's here just to show you what sometimes happens in a datawarehouse. We may need a table to keep track of various dates and special days like store closings and holidays.</li>
    </ul>
</div>